In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, power_transform
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
# Load the dataset
#from google.colab import drive
#drive.mount('/content/drive')
#df = pd.read_csv('/content/drive/My Drive/dataset/csgo_round_snapshots.csv')

#iMPORT csgo_round_snapshots.csv
from google.colab import files
df = files.upload() #uploadไฟล์

# Split X and y
y = df['round_winner']
X = df.drop(['round_winner'], axis=1)
# Drop grenade info columns
X = X.drop(X.columns[X.columns.str.contains('grenade')], axis=1)

Mounted at /content/drive


In [ ]:
# Encode categorical columns
object_cols = ['map', 'bomb_planted']
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_encoded = pd.DataFrame(ohe.fit_transform(X[object_cols]))
X_encoded.columns = ohe.get_feature_names_out(object_cols)
X_encoded.index = X.index

In [ ]:
# Combine with numerical features
X = pd.concat([X.drop(object_cols, axis=1), X_encoded], axis=1)

# Encode target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # "CT" → 0, "T" → 1

# Normalize numerical features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
# Train-test split
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X_scaled, y, stratify=y, test_size=0.1, random_state=0)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, stratify=y_train_full, test_size=0.25, random_state=0)

In [ ]:
print(X.columns.tolist())

['time_left', 'ct_score', 't_score', 'ct_health', 't_health', 'ct_armor', 't_armor', 'ct_money', 't_money', 'ct_helmets', 't_helmets', 'ct_defuse_kits', 'ct_players_alive', 't_players_alive', 'ct_weapon_ak47', 't_weapon_ak47', 'ct_weapon_aug', 't_weapon_aug', 'ct_weapon_awp', 't_weapon_awp', 'ct_weapon_bizon', 't_weapon_bizon', 'ct_weapon_cz75auto', 't_weapon_cz75auto', 'ct_weapon_elite', 't_weapon_elite', 'ct_weapon_famas', 't_weapon_famas', 'ct_weapon_g3sg1', 't_weapon_g3sg1', 'ct_weapon_galilar', 't_weapon_galilar', 'ct_weapon_glock', 't_weapon_glock', 'ct_weapon_m249', 't_weapon_m249', 'ct_weapon_m4a1s', 't_weapon_m4a1s', 'ct_weapon_m4a4', 't_weapon_m4a4', 'ct_weapon_mac10', 't_weapon_mac10', 'ct_weapon_mag7', 't_weapon_mag7', 'ct_weapon_mp5sd', 't_weapon_mp5sd', 'ct_weapon_mp7', 't_weapon_mp7', 'ct_weapon_mp9', 't_weapon_mp9', 'ct_weapon_negev', 't_weapon_negev', 'ct_weapon_nova', 't_weapon_nova', 'ct_weapon_p90', 't_weapon_p90', 'ct_weapon_r8revolver', 't_weapon_r8revolver', 'ct_

In [ ]:
# Build Neural Network model
model = keras.models.Sequential()
model.add(keras.layers.BatchNormalization(input_shape=[X_train.shape[1]]))

for _ in range(4):
    model.add(keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
    model.add(keras.layers.Dropout(rate=0.2))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/normalization/batch_normalization.py:142: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Output layer (softmax for multi-class classification)
model.add(keras.layers.Dense(2, activation="softmax"))  # 2 classes: CT, T

In [ ]:
# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',  # Categorical loss function
    optimizer='Nadam',
    metrics=['accuracy', keras.metrics.AUC(), keras.metrics.Precision(), keras.metrics.Recall()]
)

In [ ]:
# Callbacks
reduce_lr_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# แบ่งข้อมูล Train / Validation / Test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, stratify=y_train_full, test_size=0.25, random_state=0)

# สร้างโมเดล Random Forest
rf_model = RandomForestClassifier(
    n_estimators=100,  # จำนวนต้นไม้ในป่า (default = 100)
    max_depth=10,      # จำกัดความลึกของต้นไม้ (ลด overfitting)
    min_samples_split=5,  # จำนวน sample ขั้นต่ำก่อนแตก node
    min_samples_leaf=2,   # จำนวน sample ขั้นต่ำในใบไม้
    random_state=42,  # ทำให้ผลลัพธ์ reproducible
    n_jobs=-1         # ใช้ CPU ทุก core
)

# Train โมเดล
rf_model.fit(X_train, y_train)

# ประเมินผลบน Validation Set
y_pred = rf_model.predict(X_valid)
accuracy = accuracy_score(y_valid, y_pred)

print(f"Validation Accuracy: {accuracy:.4f}")

# ทดสอบโมเดลบน Test Set
y_test_pred = rf_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy:.4f}")


Validation Accuracy: 0.7699
Test Accuracy: 0.7737
